# Bike-sharing forecasting
In this tutorial we're going to forecast the number of bikes in 5 bike stations from the city of Toulouse.

In [1]:
from river import datasets, metrics, compose, preprocessing
from river_torch.regression import Regressor
import torch
from pprint import pprint
from tqdm import tqdm

In [2]:
dataset = datasets.Bikes()

for x, y in dataset:
    pprint(x)
    print(f'Number of available bikes: {y}')
    break

{'clouds': 75,
 'description': 'light rain',
 'humidity': 81,
 'moment': datetime.datetime(2016, 4, 1, 0, 0, 7),
 'pressure': 1017.0,
 'station': 'metro-canal-du-midi',
 'temperature': 6.54,
 'wind': 9.3}
Number of available bikes: 1


In [3]:
def build_fn(n_features):
    net = torch.nn.Sequential(
        torch.nn.Linear(n_features, 3),
        torch.nn.ReLU(),
        torch.nn.Linear(3, 1),
        torch.nn.Sigmoid()
    )
    return net

In [4]:
metric = metrics.MAE()
optimizer_fn = torch.optim.SGD

model_pipeline = compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind')
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= Regressor(build_fn=build_fn, loss_fn="mse", optimizer_fn=optimizer_fn)

for x, y in tqdm(dataset.take(5000)):
    y_pred = model_pipeline.predict_one(x)
    metric.update(y_true=y, y_pred=y_pred)
    model_pipeline.learn_one(x=x, y=y)
print(f'MAE: {metric.get()}')

5000it [00:02, 2227.68it/s]

MAE: 6.861629508665197


# Adaptation to Data Streams

In [ ]:
from river_torch.regression import RollingRegressor
from river_torch.utils import SequentialLSTM

In [ ]:
dataset = datasets.Bikes()

In [ ]:
def build_torch_lstm_regressor(n_features, hidden_size):
    net = torch.nn.Sequential(
        SequentialLSTM(input_size=n_features,hidden_size=hidden_size,num_layers=1),
        torch.nn.Linear(hidden_size,10),
        torch.nn.Linear(10,1)
    )
    return net

In [ ]:
metric = metrics.MAE()
optimizer_fn = torch.optim.SGD

model_pipeline = compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind')
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= RollingRegressor(
                build_fn=build_torch_lstm_regressor,
                loss_fn='mse',
                optimizer_fn=torch.optim.SGD,
                window_size=40,
                learning_rate=1e-2,
                hidden_size=1,
                num_layers=1,
                append_predict=True,
            )

for x, y in tqdm(dataset.take(5000)):
    y_pred = model_pipeline.predict_one(x)
    metric.update(y_true=y, y_pred=y_pred)
    model_pipeline.learn_one(x=x, y=y)
print(f'MAE: {metric.get()}')